# The HyperCampus 1 Algorithm

In the following discussion, I will denote the *retention index*, the desired percentage of remembered items at any point in time as specified by the user, with $\varphi$. The algorithm tries to keep memory retention at an average of $\varphi$ by scheduling cards such that they are reviewed again as soon as they are in danger of being forgotten with a probability of $1-\varphi$.

## The nature of forgetting

HyperCampus relies on the idea that forgetting is primarily caused by increased difficulty of access to a memory rather than damage to the memory itself (see [Rose et al. 2016](https://science.sciencemag.org/content/354/6316/1136) for a nice neurological study). A memory is constantly threatened by the danger of some of its access pathways being blocked by other memories. If we think in terms of discrete events that reduce accessability of a memory, we might expect that such an event happens at a constant rate. Then, the time until such an event happens is distributed according to an exponential distribution
$$p(t) = \lambda e^{-\lambda t}$$
with a rate parameter $\lambda$. Then, the share of connections that are blocked by a time $T$ is given by
$$\int_0^T p(t) dt = 1-e^{-\lambda t}.$$
Thus, the "intactness" of a memory by the time $t$ can be described as
$$\rho(t) = e^{-\lambda t}.$$
We will call $\rho$ the *retrievability* of the memory. Now there are two fundamentally different interpretations of $\rho$:
1. It indicates the *probability of recall*, meaning that if we test a person on a certain memorized item after a time $t$, the probability that they will answer correctly is $\rho(t)$. In this case, recall is binary: An item is either recalled or not. Access to it is either entirely possible with probability $\rho$ or impossible with probability $1-\rho$.
1. It is a measure of *retrieval effort*, describing how difficult it is to retrieve a memory, by the time that has to be spent on retrieving it. Access to the memory becomes progressively more difficult, as described by $\rho(t)$.

Almost all spaced repetition systems use interpretation 1. Indeed, it is empirically well documented that recall probability decays along a negative exponential in time as in the above formula, at least for simple memories. However, it is usually difficult to measure the probability of recall directly, since the result of an observation is either success or failure. When conducting a study, recall probability can be determined because a big amount of data is available, but on a trial-to-trial basis as in individual spaced repetition, this is in my eyes unreliable. Sometimes an item isn't recalled even though retrievability was high (*tip of the tongue* phenomenon), and sometimes an item is successfully recalled even though retrievability was low (by spending a lot of time trying to retrieve it).

For this reason, HyperCampus relies on the second interpretation. The continuous grade that the user provides about how difficult the retrieval was is mapped onto a retrievability estimate, irrespective of whether the item was recalled or not. We will discuss empirical evidence for this later. For now, note that both interpretations can be reconciled if access to memory is interpreted as being a statistical search: Low $\rho$ indicates difficult access to the memory (interpretation 1), meaning that the search is more likely to be aborted early (interpretation 2).

This way, HyperCampus still calculates the probability of recall as $\rho$, but $\rho$ is measured as retrieval effort. Note that the formula for $\rho(t)$ depends on an unknown rate parameter $\lambda$. Let us assume for a second that we know how to estimate $\lambda$, then the algorithm for a given item works like this:
1. Estimate $\lambda$ for that item and derive the retrievability formula $\rho(t) = e^{-\lambda t}$ for it.
1. To find the point in time when probability of forgetting rises above the $\varphi$ requested by the user, solve $\rho(T) = \varphi$ for $T$:
$$T = -\frac{\ln{\varphi}}{\lambda}$$
1. Test the user on the item after time $T$. If the grade doesn't match the expected retrievability of $1-\varphi$, update the estimate of $\lambda$ and start again at 1.

Now let us have a closer look at $\lambda$. It indicates how fast $\rho$ decays. To be precise, after a time $\Delta t = \frac{1}{\lambda}$, $\rho$ has fallen to $\frac{1}{e}$ of its original value. Obviously, $\lambda$ is different for different items stored in memory: Some remain accessible longer than others. A bit less obviously, $\lambda$ is not constant. Spaced repetition is based on the assumption that reviews can be more and more spaced out over time, indicating that $\lambda$ (the decay rate) should decrease if we have reviewed an item more often.

Thus, the difficulty lies in predicting how $\lambda$ changes with reviews. Literature indicates that effects on $\lambda$ are generally multiplicative, e.g. $\lambda$ might be halved by a review. To convert multiplication to addition, we will not deal with $\lambda$ directly but with a quantity
$$\sigma = -\ln\lambda,$$
which, following [Piotr Woźniak from SuperMemo](https://supermemo.guru/wiki/Two_component_model_of_memory), I will refer to as *stability*, as it measures how stable the storage of a memory is (however, note that SuperMemo's stability is a different but related quantity, namely $\frac{1}{\lambda}$). The stability is not constant over time, rather, it is changed by reviews depending on various factors.

## Estimating the stability

After deriving the current retrievability $\rho$ from a grade given by the user, we could update the stability estimate by simply setting
$$\widehat{\sigma} = -\ln{\left(-\frac{\ln\rho}{t}\right)}.$$
However, this would not be a very stable procedure, because the measurement of $\rho$ is supposedly not very precise. We can solve this problem by using Bayesian inference and probability distributions instead.

Let us assume we have an estimate $\widehat{\sigma}_{-}$ for the stability right before a review that occurred after an interval $t$. When we quiz the user another interval $t'$ later, we would like to know if our stability estimate was accurate and update it accordingly. To this end, we assume that there is a relationship between the observed grade $g$ and the true stability before review,
$$p(g\mid\sigma_{-}) = \mathcal{N}\left(\exp\left(-\exp\left(-\sigma_{-}\right)t'\right),\omega_g\right)=:\mathcal{N}(h_{t'}(\sigma_{-}),\omega^h),$$
meaning that the user grade equals the retrievability plus some Gaussian noise; and that the stability increase by a review after interval $t$ behaves according to
$$p(\sigma_{+}\mid\sigma_-) = \mathcal{N}(\sigma_-+f_t(\sigma_-),\omega^f)$$
(see next section for a discussion of this update). The stability supposedly remains constant between reviews.

For now, we need to assume that we actually know the stability increase function $f_t$. If that's the case, we can update our stability estimate $\widehat{\sigma}_+$ using an [extended Kalman filter](https://en.wikipedia.org/wiki/Extended_Kalman_filter). This is a Kalman filter that works for nonlinear systems by local linearization, which works well in this case because $h$ is, in fact, very well approximated by a (piecewise) linear function, and $f$ is linear, as we will see shortly.

It goes like this: Given an estimate $\widehat{\sigma}_-$ together with a variance estimate $\widehat{\Psi}^\sigma_-$ for $\widehat{\sigma}_-$, we first make a prediction for the stability after $t'$ (before the review):
$$\widehat{\sigma}_{+\mid -} = \widehat{\sigma}_- + f_t(\widehat{\sigma}_-)$$
$$\widehat{\Psi}^\sigma_{+\mid -} = f_t'^2\widehat{\Psi}^\sigma_- + \omega^f$$
with $f_t'=1+\frac{\partial f_t}{\partial \sigma}\big\rvert_{\widehat{\sigma}_-}$. When the actual grade $g$ is received after $t'$, we can calculate the residuals
$$\tilde{g} = g - h_{t'}(\widehat{\sigma}_{+\mid -})$$
$$\tilde{\Psi}^\sigma = h_{t'}'^2\widehat{\Psi}^\sigma_{+\mid -}+\omega^g$$
with $h_{t'}' = \frac{\partial h_{t'}}{\partial \sigma}\big\rvert_{\widehat{\sigma}_{+\mid -}}$, compute the approximate *Kalman gain*
$$K = \widehat{\Psi}^\sigma_{+\mid -}h_{t'}'\left(\tilde{\Psi}^\sigma\right)^{-1}$$
and use it to update the estimates
$$\widehat{\sigma}_{+} = \widehat{\sigma}_{+\mid -} + K\tilde{g}$$
$$\widehat{\Psi}^\sigma_{+} = (1-Kh'_{t'})\widehat{\Psi}^\sigma_{+\mid-}$$

This way we can update our stability estimates using the data in a Bayesian way. The only problem is now that we need to know $f_t$, i.e. we need to have a model for how the stability increases through reviews.

## The stability increase formula

A simple such model is used in the [Leitner system](https://en.wikipedia.org/wiki/Leitner_system), where a successful review doubles the interval until the next review and an unsuccessful review halves the interval. Thus, the update rule is
$$\Delta\sigma = \delta \ln 2,$$
with $\delta = 1$ for a successful review and $\delta = -1$ for an unsuccessful review.

However, there are many more factors that influence the change in stability than just whether or not the review was successful. Let us consider that memories are encoded in [*engrams*](https://en.wikipedia.org/wiki/Engram_%28neuropsychology%29), as certain cellular structures, in the brain, and that $\rho$ indicates how easily the engram can be accessed. Which factors contribute to $\sigma$?
* *synaptic strength*: how strong is the connection of the engram to the rest of the brain via its synapses?
* *complexity*: how many cells or synapses are necessary to encode the knowledge?
* *integratedness*: how well is the engram associated with other brain content in structural terms?

A common theory on the neurobiology of learning is that it is mediated by [long-term potentiation](https://en.wikipedia.org/wiki/Long-term_potentiation), which describes the phenomenon that synaptic strength increases between two cells after certain patterns of activation of the two. Thus, exposition to a fact, for example in a review, should increase the stability of the memory. Supposedly, there is a maximum possible synaptic strength, and in analogy to the [Rescorla-Wagner model](https://en.wikipedia.org/wiki/Rescorla%E2%80%93Wagner_model), I hypothesize that
$$\Delta\sigma \propto \sigma_{max} - \sigma,$$
i.e., the increase of stability slows down as $\sigma$ approaches a certain maximum possible stability $\sigma_{max}$.

*Complexity* and *integratedness* are certainly hard to quantify, but their effect on stability might be characterized. Complexity depends on the item and probably does not change significantly over time. As complex items are supposed to have a lot of connections that would have to be strengthened for an increase in stability, complexity probably cuts down stability increase by the number of required connections. Integratedness, on the other hand, can change in time facilitate stability enhancement: For example, making up a mnemonic might lead to better integration of a memory into the existing concepts, countering the effect that the complexity might have. Now because this is all very speculative, I did not try to model these considerations explicitly. Rather, I included a factor $\alpha$ in the model that scales the stability increase and is supposed to depend on the individual relationship between learner and item that can change over time.

Additional empirical evidence about the nature of stability increase comes from the *retrieval effort hypothesis*, according to which facts are learnt more efficiently when retrieval was difficult. This suggests that stability increase might depend inversely on retrievability,
$$\Delta\sigma \propto 1 - \rho.$$
Thus, we can hypothesize that the stability increase formula has the following form:
$$\Delta\sigma = \alpha(\sigma_{max}-\sigma) + \beta(1-\rho).$$

By expanding this formula, we obtain a model that is linear in $\rho$ and $\sigma$:
$$\Delta\sigma = \alpha\sigma_{max} + \beta - \alpha\sigma - \beta\rho.$$

Now, to allow for greater flexibility of the model and allowing it to include more unknown factors, we simply replace the coefficients in the above model with three independent parameters:
$$\Delta\sigma = f(\rho,\sigma) = a + b\sigma + c\rho.$$

[Evidence from SuperMemo data](https://www.supermemo.com/en/archives1990-2015/articles/stability) does indeed suggest that stability increase depends on $\rho$ and $\sigma$ in this functional way.

To cast $f$ in terms of the formalism we used for the Kalman filter, it may only depend on $\sigma$ and time. Since after an interval $t$, $\rho_{t}(\sigma) = h_{t}(\sigma)$, we may write
$$f_{t}^{\vartheta}(\sigma_t) = a + b\sigma_t + ch_{t}(\sigma_t)$$

To summarize, assuming that we have an estimate of the parameters $\widehat{\vartheta} = (a,b,c)$, our algorithm now looks like this:
1. Using the current stability estimate $\widehat{\sigma}$, find the point in time when probability of remembering decreases below the $\varphi$ requested by the user as above.
1. Test the user on the item after time $T$. Update the stability estimate through Kalman filtering, as described in the previous section, using $\widehat{\vartheta}$.
1. Using the estimated stability increase $\widehat{\sigma}_+-\widehat{\sigma}_-$ from the preceding review, update the model parameter estimate $\widehat{\vartheta}$.
1. Using the new estimate $\widehat{\vartheta}'$, calculate the new stability estimate $\widehat{\sigma}' = \widehat{\sigma} + f_t^{\widehat{\vartheta}}(\widehat{\sigma})$ and start again at 1.

Naturally, the next question is: How to find the appropriate values of $\vartheta$?

## Determining the model parameters

The problem in determining appropriate values for the model parameters $\vartheta = (a,b,c)$ is that it has to be done even when few data is available. The solution implemented by HyperCampus uses [Bayesian linear regression](https://en.wikipedia.org/wiki/Bayesian_linear_regression). The general scheme works like this:
1. Guess initial values for $\vartheta$ by specifying a prior probability distribution $p(\vartheta)$.
1. Specify a likelihood $p(\Delta\sigma\mid\vartheta)$ for how probable any observed value of the stability increase is given specific values $\vartheta$.
1. Measure $\Delta\sigma$ and use [Bayes' Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) to obtain the a probability distribution of $\vartheta$ given that observation:
$$p(\vartheta\mid\Delta\sigma) = \frac{p(\Delta\sigma)p(\Delta\sigma\mid\vartheta)}{\int p(\Delta\sigma)p(\Delta\sigma\mid\vartheta) d\Delta\sigma}$$
1. Use $p(\vartheta\mid\Delta\sigma)$ as the new prior and repeat from 3.

Since our model is linear, the problem becomes analytically tractable by using Gaussian probability densities for prior and likelihood. Let's say our prior has as mean the current estimate of the model parameters $\widehat{\vartheta}$ and some covariance $\widehat{\Psi}^{{\vartheta}}$. The likelihood mean is given by $f_t^{{\vartheta}}(\sigma,\rho) = {a}+{b}\sigma+{c}\rho$ and the variance depends on how accurately we can measure $\Delta\sigma$, which we choose to be $\omega^g+\omega^f$ since it combines the uncertainties in measuring $\rho$ with those in inferring $\sigma$. Thus:
$$p(\vartheta) = \cal{N}\left(\hat{\vartheta},\widehat{\Psi}^\vartheta\right)$$
$$p(\Delta\sigma\mid\vartheta) = \cal{N}\left(f^{\vartheta}({\sigma}),\omega^g+\omega^f\right)$$

As a well-known result, the posterior $p(\vartheta\mid\Delta\sigma) = \cal{N}\left(\hat{\vartheta}',\widehat{\Psi}'\right)$ is a Gaussian distribution with
$$\widehat{\Psi}' = \left(\widehat{\Psi}^{-1}+(\omega^g+\omega^f)^{-1}\xi(\rho,\sigma)\xi(\rho,\sigma)^{\rm T}\right)^{-1}$$
$$\widehat{\vartheta}' = \widehat{\Psi}'\left(\widehat{\Psi}^{-1}\widehat{\vartheta} + (\omega^g+\omega^f)^{-1}\Delta\sigma \xi(\rho,\sigma)\right)$$
$$\xi(\rho,\sigma) = (1,\rho,\sigma)^{\rm T}.$$

## Putting the pieces together

Now we have a procedure to infer the stability given an accurate model and another one to infer the model parameters given accurate stability estimates. How can we put them together, given that we actually know neither?

The general idea would be to infer the current stability using the Kalman filter and the current parameter estimate first, and then update the parameters using Bayesian regression and the stability estimates. However, for some time, it seemed that no matter what I tried, the algorithm wouldn't converge that way. I did find a way to improve the convergence significantly, though, based on the following two measures:
1. Repeatedly filtering the new data for a number of iterations before passing it on to the parameter regression. Because the extended Kalman filter is not an optimal estimator, for close-to-perfect estimators this actually yields slightly worse results than filtering only once, but it greatly improves convergence to the true value for estimators further away from the truth.
1. Carefully manipulating the involved variance estimators to keep the updates within sane ranges. In particular, choosing a sufficiently large $\widehat{\Psi}^\sigma$ to account for the uncertainty of the current parameters.

The final algorithm now looks like this:
1. Choose a prior for $\vartheta$ and $\sigma$.
1. Using the current stability estimate $\widehat{\sigma}$, find the point in time when probability of remembering drops below the $\varphi$ requested by the user as above. A random dispersal is added to accelerate convergence, as different values of $\rho$ can be tested.
1. Test the user on the item after time $T$. Update $\widehat{\sigma}$ by employing an extended Kalman filter for a set number of iterations and calculate the updated expected stability increase from the preceding review.
1. Perform Bayesian linear regression to find the posterior $p(\vartheta\mid\Delta\widehat{\sigma})$.
1. Using the new parameters $\widehat{\vartheta}'$, calculate the new stability.
1. Using the posterior as the new prior for $\vartheta$, repeat from 2.

The following is a quick implementation of the model in python. It is fed with noisy data generated by a process that behaves as the model predicts, with fixed parameter values $\vartheta$ saved in the variable *Th* in the code.

In [13]:
import numpy as np

## the model, uppercase = true values, lowercase = estimators
H = lambda s,t : max(min(np.exp(-np.exp(-s)*t),0.99),0.01) # forgetting curve
h = H
h_ = lambda s,t : t*np.exp(-s-t*np.exp(-s)) # derivative
F = lambda s,t : (Th @ xi(s,t).T)[0,0] # stability increase
f = lambda s,t : (th @ xi(s,t).T)[0,0]
xi = lambda s,t : np.array([[1,s,h(s,t)]])
f_ = lambda s,t : 1 + (th @ np.array([[0,1,h_(s,t)]]).T)[0,0]

RI = 0.9 # retention index
S = 2
Th = np.array([[1.0,-0.06,-0.1]]) # true parameters
th = np.array([[0.7,-0.08,-0.1]])

Psith = 0.3*np.eye(3)
Psith[1,1] = 0.03
Psith[2,2] = 0.03

s = 3.6
Psis = 0.83
of = 0.83 # modelling uncertainty
og = 0.2 # measurement noise

t = 1 # interval before last
T = 4 # last interval
sums = 0
sumph = 0
def sim(r) :
    global t,T,sums,sumph,S,s,Psith,th,i,Psis
    _S = S
    S = S + F(S,t) + np.random.normal(0,of)
    R = H(S,T)

    _s = s
    s = s + f(_s,t)
    Psis = Psis*f_(_s,t)**2 + of**2

    for j in range(0,15) :
        dr = r - h(s,T)
        dp = Psis*h_(s,T)**2 + og**2
        K = Psis*h_(s,T)*1/max(dp,0.01)
        s = s + K*dr
        Psis = (1-K*h_(s,T))*Psis

    invPsith = np.linalg.inv(Psith)
    Psith = np.linalg.inv(invPsith + 1/(of**2+og**2) * xi(_s,t).T @ xi(_s,t))
    th = (Psith @ ((invPsith @ th.T) + 1/(of**2+og**2) * (s-_s) * xi(_s,t).T)).T

    s_ = s + f(s,T) if f(s,T) >= 0 else s # force stability increase to be non-negative
    print(" expected next sigma %f true %f" % (s_,S + F(S,T)))
    t = T
    T = int(-np.log(RI)*np.exp(s_)*(0.9+np.random.random()*0.2)) + 1
    print(" next interval %f optimal %f" % (T,-np.log(RI)*np.exp(S + F(S,T))))

for i in range(1,10) :
    r = H(S+F(S,t),T) + np.random.normal(0,0.1)
    sim(r)
    print("#" + str(i) + " theta " + str(th))

print("true theta" + str(Th))

 expected next sigma 3.187912 true 3.914104
 next interval 3.000000 optimal 5.260218
#1 theta [[ 0.5454384  -0.13564218 -0.11503956]]
 expected next sigma 4.918069 true 4.923107
 next interval 16.000000 optimal 14.711066
#2 theta [[ 0.76311159 -0.07466684 -0.09902051]]
 expected next sigma 5.712567 true 5.822052
 next interval 30.000000 optimal 35.813606
#3 theta [[ 0.78769779 -0.05367815 -0.09616725]]
 expected next sigma 7.327651 true 5.444989
 next interval 166.000000 optimal 25.714258
#4 theta [[ 0.80964931 -0.00624518 -0.09445163]]
 expected next sigma 5.685094 true 6.579747
 next interval 30.000000 optimal 73.917357
#5 theta [[ 0.86263284 -0.10490884 -0.09106381]]
 expected next sigma 6.733080 true 8.637882
 next interval 82.000000 optimal 595.067812
#6 theta [[ 0.88362143 -0.08174955 -0.10121393]]
 expected next sigma 9.196801 true 9.146183
 next interval 985.000000 optimal 1000.967643
#7 theta [[ 0.84362697 -0.01831062 -0.09840565]]
 expected next sigma 8.896865 true 9.504653
 

As you can see, the model takes some time to find good estimates in the noisy environment, and the convergence isn't perfect. It does react to deviations from its expectations in the way one would want it to, but the actual usefulness of the model for studying still needs to be evaluated.

## What happens in case of a lapse?

The above version of the algorithm does not differentiate between successful and unsuccessful reviews, since the retrievability is determined from the retrieval effort grading alone. Intuitively, it seems that items that were forgotten should be reviewed again after a short interval. Empirical evidence on this is inconclusive:
* Retrieval attempts enhance learning regardless of whether they were successful or not. Attempting to retrieve an item from memory and subsequent activation of it seems to strenghten the relevant connections, whether the activation was triggered from "inside" (by finding it on one's own) or "outside" (by being told the solution).
* Retrieval effort correlates with gain of memory strength.

Now the interesting question would be the combination of the two: Does the gain of learning in unsuccessful retrieval attempts correlate with retrieval effort? I couldn't find any study that addressed this question. Also, none of the studies I have seen on this so far have investigated whether these effects also hold true for long intervals (such as months or years). Therefore, it might not be wise to treat lapses entirely on the basis of these findings.

HyperCampus will try to find out about this question by evaluating user performance for successful and unsuccessful retrieval attempts separately. In practice, this means that there are two different $\beta$ parameters for the model, i.e. the parameter controlling the influence of $\rho$ on the stability increase, that the algorithm uses in its inference process. Future evaluation of the converged parameters might help to reveal the quantitative role of memory retrieval success in learning.

In my eyes, spaced repetition works well for consolidating items in memory, but not so well for learning them initially. This may also apply to relearning lapsed cards. Thus, the default mode of the algorithm is to relay the responsibility for learning and relearning items to the user (but, see the [user guide](user_guide.md) for some options to facilitate acquisition).

## An aside: Expectation-maximization

Instead of the Bayesian update of the model parameters, one might do [expectation-maximization](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm#Filtering_and_smoothing_EM_algorithms). This is what I tried to do first, and it didn't work as well, but for documentation in case I want to pursue this idea further another time, here is the maximum likelihood estimator for $\vartheta$ that I have come up with, given data $(\widehat{\sigma}_1,\ldots,\widehat{\sigma}_n)$:

$$\vartheta^*=\left(\sum_{i=2}^n\xi(\widehat{\sigma}_{i-1})\xi^{\rm T}(\widehat{\sigma}_{i-1})\right)^{-1}\sum_{i=2}^n (\widehat{\sigma}_i-\widehat{\sigma}_{i-1})\xi(\widehat{\sigma}_{i-1})$$